# Synchronization

## Setup and Helper Functions

In [ ]:
import re
import json

from pathlib import Path

import pandas as pd
import numpy as np
from scipy import signal

from fau_colors import cmaps
import biopsykit as bp

from nilspodlib import Dataset
from biopsykit.io.biopac import BiopacDataset
from empkins_io.sensors.emrad import EmradDataset
from empkins_io.sync import SyncedDataset
from packaging.version import Version

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

In [ ]:
data_path = Path("data")

radar_path = data_path.joinpath("data_sync-test-02.h5")
biopac_path = data_path.joinpath("data_sync-test-02.acq")
nilspod_path = sorted(data_path.glob("*.bin"))[1]

## Load Data

### Radar

In [ ]:
dataset_radar = EmradDataset.from_hd5_file(radar_path)
data_radar = dataset_radar.data_as_df(index="local_datetime", add_sync_in=True, add_sync_out=True)
data_radar = data_radar.xs("rad1", axis=1)

data_radar.head()

### NilsPod

In [ ]:
dataset_nilspod = Dataset.from_bin_file(
    nilspod_path, legacy_support="resolve", force_version=Version("0.17.0"), tz="Europe/Berlin"
)
data_nilspod = dataset_nilspod.data_as_df(index="local_datetime")
data_nilspod.head()

### Biopac

In [ ]:
dataset_biopac = BiopacDataset.from_acq_file(biopac_path)
data_biopac = dataset_biopac.data_as_df(index="local_datetime")

data_biopac.head()

In [ ]:
synced_dataset = SyncedDataset(sync_type="peak")

synced_dataset.add_dataset(
    "nilspod", data_nilspod, sync_channel_name="analog_1", sampling_rate=dataset_nilspod.info.sampling_rate_hz
)
# synced_dataset.add_dataset(
#    "biopac", data_biopac, sync_channel_name="sync", sampling_rate=dataset_biopac.sampling_rate_hz_sync
# )
synced_dataset.add_dataset(
    "radar", data_radar, sync_channel_name="Sync_In", sampling_rate=dataset_radar.sampling_rate_hz
)

In [ ]:
fig, axs = synced_dataset.plot_sync_channels(figsize=(8, 3))
fig.tight_layout()

In [ ]:
synced_dataset.cut_to_sync_start()

In [ ]:
data_nilspod_cut = synced_dataset.nilspod_cut_
data_radar_cut = synced_dataset.radar_cut_

fig, ax = plt.subplots(figsize=(8, 2))
ax2 = ax.twinx()
data_radar_cut[["Sync_In"]].plot(ax=ax)
# data_biopac[["sync"]].plot(ax=ax)
data_nilspod_cut.filter(like="analog_1").plot(ax=ax2, color="g")
fig.tight_layout()

In [ ]:
synced_dataset.align_datasets("nilspod", cut_to_shortest=True)

In [ ]:
data_nilspod_aligned = synced_dataset.nilspod_aligned_
data_radar_aligned = synced_dataset.radar_aligned_

fig, ax = plt.subplots(figsize=(8, 3))
ax2 = ax.twinx()
data_radar_aligned[["Sync_In"]].plot(ax=ax)
# data_biopac[["sync"]].plot(ax=ax)
data_nilspod_aligned.filter(like="analog_1").plot(ax=ax2, color="g")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
data_radar[["Sync_In"]].plot(ax=ax)
data_biopac[["sync"]].plot(ax=ax)
data_nilspod.filter(like="analog_1").plot(ax=ax2, color="g")
fig.tight_layout()